In [1]:
import tensorflow as tf

print(tf.__version__)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

1.6.0-rc1
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
#Optimization variables

learning_rate = 0.0001
epochs = 10
batch_size = 50
beta = 0.01 #regularizer

#training data placeholders
# input x = 28 x 28 pixels = 784 flattened image

#mnist.train.nextbatch()

x = tf.placeholder(tf.float32, [None, 784])

#dynamically reshape the input

x_shaped = tf.reshape(x, [-1, 28, 28, 1])

#output data placeholder

y = tf.placeholder(tf.float32, [None, 10])

In [3]:
#function to create layers for model

def create_new_conv_layer(input_data, num_input_channels, num_filters, filter_shape, pool_shape, name):
    
    #make the filter input shape for tf.nn.conv_2d
    
    conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels, num_filters]
    
    #initialization of w and b for the filter
    
    weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev = 0.03), name= name+'_W')
    bias = tf.Variable(tf.truncated_normal([num_filters]), name = name+'_b')
    
    #setup conv layer operation
    
    out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding = "SAME")
    
    #adding the bias
    
    out_layer += bias
    
    #relu activation
    
    out_layer = tf.nn.relu(out_layer)
    
    #max-pooling
    
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    strides = [1, 2, 2, 1]
    out_layer = tf.nn.max_pool(out_layer, ksize = ksize, strides = strides, padding = "SAME")
    
    return out_layer

In [4]:
#create some conv layers

layer1 = create_new_conv_layer(x_shaped, 1, 32, [5, 5], [2, 2], name = 'layer1')
layer2 = create_new_conv_layer(layer1, 32, 64, [5, 5], [2, 2], name = 'layer2')

In [5]:
#FC layer

flattened = tf.reshape(layer2, [-1, 7*7*64])

In [6]:
#setup some weights and bias values for this layer, then activate with ReLU

wd1 = tf.Variable(tf.truncated_normal([7*7*64, 1000], stddev = 0.03), name='wd1')
bd1 = tf.Variable(tf.truncated_normal([1000], stddev = 0.01), name='bd1')
dense_layer1 = tf.matmul(flattened, wd1) + bd1
dense_layer1 = tf.nn.relu(dense_layer1)

In [7]:
#Another layer with softmax activation

wd2 = tf.Variable(tf.truncated_normal([1000, 10], stddev = 0.03), name = 'wd2')
bd2 = tf.Variable(tf.truncated_normal([10], stddev = 0.01), name = 'bd2')
dense_layer2 = tf.matmul(dense_layer1, wd2) + bd2
y_ = tf.nn.softmax(dense_layer2)


regularizers = tf.nn.l2_loss(wd1) + tf.nn.l2_loss(wd2)

In [ ]:
#Optimization function 

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=dense_layer2, labels=y) + beta * regularizers)

In [ ]:
#Optimizer

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cross_entropy)

#Accuracy

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#Setup the initialization operator

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    # initialise the variables
    sess.run(init_op)
    total_batch = int(len(mnist.train.labels) / batch_size)
    writer = tf.summary.FileWriter("output", sess.graph)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            _, c = sess.run([optimizer, cross_entropy], 
                            feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        test_acc = sess.run(accuracy, 
                       feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost), " test accuracy: {:.3f}".format(test_acc))

    print("\nTraining complete!")
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))
    writer.close()

Epoch: 1 cost = 3.401  test accuracy: 0.925
Epoch: 2 cost = 0.625  test accuracy: 0.950
Epoch: 3 cost = 0.336  test accuracy: 0.966
Epoch: 4 cost = 0.239  test accuracy: 0.972
Epoch: 5 cost = 0.194  test accuracy: 0.977
Epoch: 6 cost = 0.168  test accuracy: 0.981
Epoch: 7 cost = 0.151  test accuracy: 0.981
Epoch: 8 cost = 0.138  test accuracy: 0.982


# Train time - 6mins 34secs
# Proc - i5-7700HQ
# gpu - 1050ti
# Py - 3.6.4
# Tensorflow - 1.6.0-rc1 gpu
# Cuda Toolkit - 9.0